In [1]:
import time, datetime
import copy
import os
import sys
import warnings
import torch
import numpy as np
from loguru import logger
import yaml
from utils import dumb_reward_plot
import gym
os.environ["CUDA_VISIBLE_DEVICES"]="1"
sys.path.append('./envs/cartpole-envs')
sys.path.append('./')
import cartpole_envs

from utils import plot_reward, plot_index
from mpc.mpc_cp import MPC
from baselines.NP_epi import NP

def prepare_dynamics(gym_config):
    dynamics_name = gym_config['dynamics_name']
    seed = gym_config['seed']
    dynamics_set = []
    for i in range(len(dynamics_name)):
        dynamics_set.append(gym.make(dynamics_name[i]))
    task = [dynamics_set[i] for i in gym_config['task_dynamics_list']]
    return task

def load_config(config_path="config.yml"):
    if os.path.isfile(config_path):
        f = open(config_path)
        return yaml.load(f, Loader=yaml.FullLoader)
    else:
        raise Exception("Configuration file is not found in the path: "+config_path)

In [2]:
# config = load_config('config/config_cpstable_np.yml')
config = load_config('config/config_swingup_robust.yml')
mpc_config = config['mpc_config']
gym_config = config['gym_config']
render = gym_config['render']
np_config = config['NP_config']

model = NP(NP_config=np_config)
model.model.load_state_dict(torch.load( './misc/log/robust_model_latent-10-09-22.pth'))

logger.info('Using model: {}', model.name)

mpc_controller = MPC(mpc_config=mpc_config)

# prepare task
task = prepare_dynamics(gym_config)
# print(gym_config)

"""start DPGP-MBRL"""
data_buffer = []
label_list = []
subtask_list = []
subtask_reward = []
subtask_succ_count = [0]
comp_trainable = [1]
task_reward = []
trainable = True
task_solved = False
subtask_solved = [False, False, False, False]
total_count = 0
task_epi = 0
log_name = None

total_tasks = 1

2020-07-10 10:34:53.904 | INFO     | __main__:<module>:11 - Using model: NP


In [3]:
config = load_config('config/config_swingup_robust.yml')
mpc_config = config['mpc_config']
mpc_controller = MPC(mpc_config=mpc_config)
"""testing the model with MPC while training """
test_episode = 1
test_epoch = 10
log = []
m_p_list = [0.3, 0.4, 0.5, 0.6, 0.7]
l_list = [0.3, 0.4, 0.5, 0.6, 0.7]

for m_p in m_p_list:
    for l in l_list:
        task_steps = 0
        env = task[0]
#         m_p = m_p_list[np.random.randint(2)]
#         l = l_list[np.random.randint(2)]
        env.unwrapped.m_p = m_p
        env.unwrapped.l = l
        for epi in range(test_episode):
            acc_reward = 0
            obs = env.reset()
            O, A, R, acc_reward, done, V = [], [], [], 0, False, []
            mpc_controller.reset()
            i = 0
            while not done:
                i+= 1
                env_copy = prepare_dynamics(gym_config)[0]
                env_copy.unwrapped.m_p = m_p
                env_copy.unwrapped.l = l
                env_copy.reset()
                if task_steps > 0:
                    action = np.array([mpc_controller.act(task=env_copy, model=model, state=obs, ground_truth=True)])
                else:
                    action = np.array([0.0])
                obs_next, reward, done, violation = env.step(action)
                task_steps += 1
                A.append(action)
                O.append(obs_next)
                R.append(reward)
                V.append(violation)

                model.data_process([0, obs, action, obs_next - obs])
                obs = obs_next
                acc_reward += reward
#             print('task: ', task_idx,'step: ', i, 'acc_reward: ', acc_reward, 'violation_rate: ', sum(V)/len(V))
            print('pole_mass: ', m_p, 'pole_length: ', l, 'step: ', i, 'acc_reward: ', acc_reward, 'violation_rate: ', sum(V)/len(V))
            env.close()

            if done:
                samples = {
                    "obs": np.array(O),
                    "actions": np.array(A),
                    "rewards": np.array(R),
                    "reward_sum": acc_reward,
                    "violation_rate": sum(V)/len(V)
                }
                log.append(samples)
#                 if log_name is None:
#                     log_name = datetime.datetime.now()
#                 path = './misc/log/np_adaptation' + log_name.strftime("%d-%H-%M") + '.npy'
#                 np.save(path, log, allow_pickle=True)
#                 dumb_reward_plot(path)
            
        model.reset()

tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9572, -2.1486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor

tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.1267, -2.3514]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor

tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.3495, -2.6179]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor

tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.5953, -2.9118]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor

tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 2.7636, -3.1132]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor

In [ ]:
z1 